## 语言模型的计算
假设一段长度为$T$的文本中的词依次为$w_1, w_2, \ldots, w_T$，那么在离散的时间序列中，$w_t$（$1 \leq t \leq T$）可看作在时间步（time step）$t$的输出或标签。给定一个长度为$T$的词的序列$w_1, w_2, \ldots, w_T$，语言模型将计算该序列的概率：

$$P(w_1, w_2, \ldots, w_T) = \prod_{t=1}^T P(w_t \mid w_1, \ldots, w_{t-1}).$$

- 也就是$w_1$~$w_T$的句子概率为第一个的概率乘（在第一个出现的条件下，第二个词出现的概率）
- 统计学上，$P（w_1）$ 是$w_1$在数据集中出现的次数除以总词数
- $P(w_2 \mid w_1)$可以计算为$w_1, w_2$两词相邻的频率与$w_1$词频的比值，因为该比值即$P(w_1, w_2)$与$P(w_1)$之比

> 但是右式化简得到左式，哪为什么不直接算左式
> - 因为左式是“一个整体”的 joint probability，几乎没数据支撑，直接统计没法用；右式是“拆成一串局部条件概率”，可以用 n-gram 统计 + 平滑来估计；

## $n$元语法

> 你走先，你先走，一元和二元计算的概率都没区别吧,在数据集中，走先，先走出现概率不一样

> P(你先走) = P(你 |) · P(先 | 你) · P(走 | 先)

> P(你走先) = P(你 |) · P(走 | 你) · P(先 | 走)


## RNN

RNN 相比于 MLP 的本质区别是：在每个时间步多了一个“隐含状态 
$𝐻_t$"的递归依赖，同时参数在时间维上共享，从而可以一边处理当前输入，一边“记住”之前的信息。
在最基础的 RNN 单元里，一步的计算可以抽象成：

> $\textbf{Input: } (X_t,\; H_{t-1})$
- 第一步时，$H_0$随机初始化

> $\textbf{隐藏变量: }H_t = \phi\left( X_t W_{xh} + H_{t-1} W_{hh} + b_h \right)$<br>
> $\textbf{Output: } O_t = H_t W_{hy} + b_y$
$$
\underbrace{[\,X_t,\; H_{t-1}\,]}_{\text{拼接向量}}
\;
\underbrace{
\begin{bmatrix}
W_{xh} \\[4pt]
W_{hh}
\end{bmatrix}
}_{\text{拼接矩阵}}
\;=\;
X_t W_{xh} + H_{t-1} W_{hh}
$$

## RNN 语言模型公式与直观理解小结

### 1. RNN 状态递推

给定输入序列的向量表示 $X_t$（第 $t$ 个词的 embedding）：

$$
H_t = \phi\left( X_t W_{xh} + H_{t-1} W_{hh} + b_h \right)
$$

- $H_t$：第 $t$ 步的隐藏状态（隐含向量）  
- $\phi(\cdot)$：非线性激活函数（如 $\tanh$、ReLU 等）  
- $W_{xh}, W_{hh}, b_h$：可学习参数  

展开递推可以写成：

$$
H_t = F_\theta(X_1, X_2, \dots, X_t)
$$

也就是说，**$H_t$ 是对前 $t$ 个输入的“线性变换 + 非线性叠加”后的压缩表示**。

---

### 2. 语言模型的概率分解

对词序列 $(w_1,\dots,w_T)$，语言模型要刻画的是联合概率：

$$
P(w_1, \dots, w_T)
= \prod_{t=1}^{T} P\big(w_t \mid w_1,\dots,w_{t-1}\big)
$$

---

### 3. 用 RNN 近似条件概率

在第 $t-1$ 个时间步，RNN 的隐藏状态为 $H_{t-1}$。  
用一个线性层 + softmax 给出“下一个词 $w_t$”的条件概率：

$$
\text{logits}_{t-1} = H_{t-1} W_{hp} + b_p
$$

$$
P_\theta(w_t = i \mid H_{t-1})
= \mathrm{softmax}(\text{logits}_{t-1})_i
$$

关键假设：

$$
P_\theta(w_t \mid w_1,\dots,w_{t-1})
\approx
P_\theta(w_t \mid H_{t-1})
=
\mathrm{softmax}(H_{t-1} W_{hp} + b_p)_{w_t}
$$

于是整个序列的概率可以写成：

$$
P_\theta(w_1,\dots,w_T)
\approx
\prod_{t=1}^{T}
\mathrm{softmax}(H_{t-1} W_{hp} + b_p)_{w_t}
$$

---

### 4. 损失函数（负对数似然 / 交叉熵）

训练时最小化负对数似然：

$$
\mathcal{L}(\theta)
= - \sum_{t=1}^{T}
\log P_\theta(w_t \mid w_1,\dots,w_{t-1})
\approx
- \sum_{t=1}^{T}
\log \mathrm{softmax}(H_{t-1} W_{hp} + b_p)_{w_t}
$$

---

### 5. 你的直观理解（整理版）

- **$H_t$**：可以看作对当前输入 $X_t$ 和所有历史输入 $(X_1,\dots,X_{t-1})$ 进行“线性变换 + 非线性叠加”得到的压缩表示，是一个**固定维度的历史信息摘要**。

- 我们 **假设** 存在参数 $(W_{hp}, b_p)$，使得  
  $H_{t-1} W_{hp} + b_p$ 经过 softmax 后，**可以近似刻画**  
  “在前 $t-1$ 个词给定条件下，第 $t$ 个词的条件概率分布”。

- **概率模型是一种描述方式**：  
  链式法则 + softmax 把 RNN 的输出解释成条件概率。

- **内部仍是黑盒表示学习过程**：  
  RNN 如何在 $H_t$ 内编码历史信息是网络自动学的，我们只知道  
  $H_t$ 是“关于历史的某种压缩特征”，但无法解析每一维具体含义。
